## Install dependencies

### Install python

In [ ]:
%%capture
!apt-get update
!apt install python3.10
!apt install python3.11

### Install poetry

In [ ]:
%%capture
!pip install poetry

### Clone the repository

In [ ]:
!git clone https://github.com/WIMU-BKT/WIMU-Data-poisoning.git

Cloning into 'WIMU-Data-poisoning'...
remote: Enumerating objects: 503, done.
remote: Counting objects: 100% (271/271), done.
remote: Compressing objects: 100% (213/213), done.
remote: Total 503 (delta 93), reused 169 (delta 39), pack-reused 232 (from 1)
Receiving objects: 100% (503/503), 12.85 MiB | 14.85 MiB/s, done.
Resolving deltas: 100% (183/183), done.


In [ ]:
import os

os.chdir("./WIMU-Data-poisoning")

### Intialize submodules

In [ ]:
!git checkout nightshade
!git submodule init
!git submodule update

Branch 'audioldm-finetuning' set up to track remote branch 'audioldm-finetuning' from 'origin'.
Switched to a new branch 'audioldm-finetuning'
Submodule 'wimudp/data_poisoning/finetuning/audioldm' (https://github.com/mateusztobiasz/AudioLDM-finetuning) registered for path 'wimudp/data_poisoning/finetuning/audioldm'
Cloning into '/content/WIMU-Data-poisoning/wimudp/data_poisoning/finetuning/audioldm'...
Submodule path 'wimudp/data_poisoning/finetuning/audioldm': checked out 'c9eaca0641d3443561de28640cea5acaec253685'


### Create poetry venv and install dependencies

In [ ]:
!poetry env use python3.11 && poetry install

Streaming output truncated to the last 5000 lines.
  - Installing stack-data (0.6.3): Pending...
  - Installing tokenizers (0.13.3): Pending...
  - Installing torch (2.5.1+cu121): Pending...
  - Installing matplotlib-inline (0.1.7)
  - Installing mdit-py-plugins (0.3.3)
  - Installing orjson (3.10.11)
  - Installing pandas (2.2.3): Downloading... 8%
  - Installing pexpect (4.9.0)
  - Installing prompt-toolkit (3.0.48)
  - Installing psutil (6.1.0)
  - Installing pydub (0.25.1)
  - Installing pygments (2.18.0): Downloading... 70%
  - Installing python-multipart (0.0.17)
  - Installing pyzmq (26.2.0): Downloading... 50%
  - Installing regex (2024.11.6): Installing...
  - Installing safetensors (0.4.5): Downloading... 100%
  - Installing semantic-version (2.10.0)
  - Installing stack-data (0.6.3): Pending...
  - Installing tokenizers (0.13.3): Pending...
  - Installing torch (2.5.1+cu121): Pending...
  - Installing matplotlib (3.7.0): Downloading... 10%
  - Installing matplotlib-inline (0

### Install AudioLDM dependencies

In [ ]:
import os
os.chdir("./wimudp/data_poisoning/finetuning/audioldm")

In [ ]:
!git checkout main
!git pull --rebase

Previous HEAD position was c9eaca0 Prepared dataset for fine tuning
Switched to branch 'main'
Your branch is up to date with 'origin/main'.
Already up to date.


In [ ]:
!poetry env use python3.10 && poetry install

Streaming output truncated to the last 5000 lines.
  - Installing torchlibrosa (0.0.9)
  - Installing torchmetrics (1.2.0): Installing...
  - Installing torchvision (0.14.1): Downloading... 0%
  - Installing tqdm (4.66.1)
  - Installing wave (0.0.2): Preparing...
  - Installing prompt-toolkit (3.0.41): Downloading... 0%
  - Installing pygments (2.16.1): Downloading... 100%
  - Installing pyyaml (6.0.1)
  - Installing scikit-image (0.22.0): Downloading... 10%
  - Installing stack-data (0.6.3)
  - Installing torchlibrosa (0.0.9)
  - Installing torchmetrics (1.2.0): Installing...
  - Installing torchvision (0.14.1): Downloading... 0%
  - Installing tqdm (4.66.1)
  - Installing wave (0.0.2): Preparing...
  - Installing pyyaml (6.0.1)
  - Installing scikit-image (0.22.0): Downloading... 10%
  - Installing stack-data (0.6.3)
  - Installing torchlibrosa (0.0.9)
  - Installing torchmetrics (1.2.0): Installing...
  - Installing torchvision (0.14.1): Downloading... 0%
  - Installing tqdm (4.66.1

## Mounting Google Drive and moving preloaded audio files

In [ ]:
import os

from google.colab import drive

drive.mount('/content/drive')
os.chdir("../..")

In [ ]:
!tar -xf /content/drive/MyDrive/night_audios.tar.xz -C ./data

## Filtering dataset, extracting data and generating poison samples

In [ ]:
import os

os.chdir("./dirty_label")

In [ ]:
!poetry run python dataset_filter.py

In [ ]:
import os

os.chdir("../nightshade")

In [ ]:
!poetry run python data_extractor.py
!poetry run python poison_generator.py

## Finetuning AudioLDM

### Moving required files

In [ ]:
import os
os.chdir("../finetuning/audioldm")

In [ ]:
%%bash
mv -t ./data/checkpoints /content/drive/MyDrive/audiomae_16k_128bins.ckpt \
/content/drive/MyDrive/clap_htsat_tiny.pt \
/content/drive/MyDrive/clap_music_speech_audioset_epoch_15_esc_89.98.pt \
/content/drive/MyDrive/hifigan_16k_64bins.ckpt \
/content/drive/MyDrive/hifigan_16k_64bins.json \
/content/drive/MyDrive/hifigan_48k_256bins.ckpt \
/content/drive/MyDrive/hifigan_48k_256bins.json \
/content/drive/MyDrive/vae_mel_16k_64bins.ckpt \
/content/drive/MyDrive/audioldm-s-full

### Restructure dataset and create metadata files

In [ ]:
!poetry run python3 data_script.py

Dataset structured successfully!


### Create xterm console and run finetuning script


In [ ]:
!pip install colab-xterm
%load_ext colabxterm
%xterm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 10.1 MB/s eta 0:00:00


Launching Xterm...

<IPython.core.display.Javascript object>

In [ ]:
!poetry run python3 audioldm_train/train/latent_diffusion.py \
-c audioldm_train/config/2023_08_23_reproduce_audioldm/audioldm_original.yaml \
--reload_from_ckpt data/checkpoints/audioldm-s-full

### Inference with best model

In [ ]:
!poetry run python3 audioldm_train/infer.py \
--config_yaml ./audioldm_train/config/2023_08_23_reproduce_audioldm/audioldm_original.yaml \
--list_inference ./tests/captionlist/inference_test.lst \
--reload_from_ckpt "./log/latent_diffusion/2023_08_23_reproduce_audioldm/audioldm_original/checkpoints/checkpoint-fad-val/frechet_inception_distance=133.00-global_step=global_step=889.ckpt"